# Shifting Previous Week Stats to Predict Current Week Performance
Author: Aidan O'Connor   
Date: 15 June 2021   

In this notebook, I'll take previous week stats and shift them to current week predictions.

In [1]:
# Import pandas for data manipulation and sqlite3 for stored data access
import pandas as pd
import sqlite3

In [2]:
# Create a database connection
conn = sqlite3.connect('../../fixtures/database/cloudy_with_a_chance_of_football.db')
cursorObj = conn.cursor()

In [5]:
# Read in Defense stats and Red Zone stats
defense = pd.read_csv('../../fixtures/cleaned_data/fantasyDefenseScores.csv')
redzone = pd.read_csv('../../fixtures/cleaned_data/red_zone_stats.csv')

# Read in stats data grouped and ordered by PlayerID and week_id
stats = pd.read_sql_query(
    """
    SELECT PlayerID,new_week_id
    from stats_regular
    GROUP BY PlayerID, new_week_id
    ORDER BY PlayerID, new_week_id;
    """
    ,conn
).drop_duplicates()

In [38]:
# Split defense into 2019 and 2020 dataframes
defense_2019 = defense[defense['Season'] == 2019]
defense_2020 = defense[defense['Season'] == 2020]

# Split redzone into 2019 and 2020 dataframes
redzone_2019 = redzone[redzone['Season'] == 2019]
redzone_2020 = redzone[redzone['Season'] == 2020]

def_red_frames = [defense_2019,defense_2020,redzone_2019,redzone_2020]

In [51]:
for n in def_red_frames:
    #n['PlayerID_x'] = n.PlayerID
    #n.set_index('PlayerID_x', inplace = True)
    n = n.sort_values(by = ['PlayerID','Week'], ascending = True, inplace = True)

<ipython-input-51-ace5ffcc01e1>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  n = n.sort_values(by = ['PlayerID','Week'], ascending = True, inplace = True)


In [45]:
# Create year and week columns to help order the data, then sort by both
stats['year'] = [n[0:4] for n in stats['week_id']]
stats['week'] = [int(n[-1]) if len(n) == 6 else int(n[-2:]) for n in stats['week_id']]
stats = stats.sort_values(by = ['PlayerID','week'])

# Create 2 subordinate dataframes to ensure no data overlap
stats_2019 = stats[['PlayerID','week']][stats['year'] == '2019']
stats_2020 = stats[['PlayerID','week']][stats['year'] == '2020']

# Make a list of dataframes to apply this next function to
frames_to_shift = [stats_2019,stats_2020]

KeyError: 'week_id'

In [46]:
def stat_shifter(dataframe):
    """
    Shifts previous week stats to help predict current week performance
    Different from pandas shift, takes into account unique player ID
    Input: dataframe with PlayerID and week columns
    """
    new_week_id_list = []
    
    for n in range(0,len(dataframe) - 1):
        if dataframe['PlayerID'].iloc[n] == dataframe['PlayerID'].iloc[n+1]:
            new_week_id_list.append(dataframe['week'].iloc[n+1])
        else:
            new_week_id_list.append(0)
            
    new_week_id_list.append(0)
    dataframe['new_week_id_list'] = new_week_id_list

    return dataframe

In [ ]:
# Apply the stat_shifter function to the dataframes
for n in frames_to_shift:
    n = stat_shifter(n)

In [52]:
# Apply the stat_shifter function to the redzone and defense dataframes
for n in [defense_2019,defense_2020,redzone_2019,redzone_2020]:
    n = n.rename({'Week':'week'}, axis = 'columns')
    n = stat_shifter(n)

In [53]:
# Append 2020 dataframe to 2019 dataframe for all three sets of dataframes
merged_redzone = redzone_2019.append(redzone_2020)
merged_defense = defense_2019.append(defense_2020)
merged_stats = stats_2019.append(stats_2020)

In [ ]:
# Create a week_id and new_week_id column for each subordinate dataframe
stats_2019['week_id'] = '2019_' + stats_2019['week'].astype(str)
stats_2019['new_week_id'] = '2019_' + stats_2019['new_week_id_list'].astype(str)
stats_2020['week_id'] = '2020_' + stats_2020['week'].astype(str)
stats_2020['new_week_id'] = '2020_' + stats_2020['new_week_id_list'].astype(str)

In [ ]:
# Read in stats data
df = pd.read_csv('../../fixtures/cleaned_data/fantasyPlayerScores.csv')

In [ ]:
# Merge the shifted stats columns and the overall stats table, dropping 2018
# from the merged dataframe
merged_df = pd.merge(
    df,
    merged_stats.drop(['week','new_week_id_list'], axis = 'columns'),
    how = 'left',
    left_on = ['PlayerID','week_id'],
    right_on = ['PlayerID','week_id']
)

merged_df = merged_df[merged_df['Season'] != 2018].drop('week_id', axis = 'columns')

In [ ]:
# Drop the old stats table...
cursorObj.execute("DROP TABLE stats_regular")

In [ ]:
# ...and add in the new one, then check to make sure it stuck
merged_df.to_sql('stats_regular',
                con = conn,
                index = False,
                if_exists = 'append'
)

cursorObj.execute('SELECT name from sqlite_master where type = "table"')
print(cursorObj.fetchall())

In [54]:
merged_redzone.to_sql('redzone_stats',
                     con = conn,
                    index = False,
                    if_exists = 'append'
)

merged_defense.to_sql('defense_stats',
                     con = conn,
                    index = False,
                    if_exists = 'append'
)

cursorObj.execute('SELECT name from sqlite_master where type = "table"')
print(cursorObj.fetchall())

[('positions',), ('player_dob',), ('players',), ('player_team',), ('player_age',), ('team_home_time_zone',), ('injury_status',), ('days_since_last_game',), ('weather',), ('hours_displaced',), ('stats_red_zone',), ('elevation_and_conference',), ('performance_target',), ('stats_regular',), ('redzone_stats',), ('defense_stats',)]


In [55]:
# Close the database connection
conn.close()